# Notebook 2: Data Processing & Vector Database

**Objectives:**
- Clean and structure event data
- Use LLM to extract additional metadata (baby-friendliness only)
- Generate embeddings for semantic search
- Set up Qdrant and index all events

**✅✅✅ Important Notes:**
- We're keeping it simple: only extracting `baby_friendly: bool` from LLM
- Semantic search will naturally handle mood/vibe queries
- Event-level chunks (1 event = 1 document)

---


## Setup & Imports


In [1]:
# Install required packages if needed
# !pip install openai qdrant-client pandas numpy python-dotenv


In [16]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), organization=os.getenv("OPENAI_ORG_ID"))

print("✅ Imports successful!")
print(f"OpenAI API Key loaded: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")


✅ Imports successful!
OpenAI API Key loaded: Yes


## 1. Load Raw Data

Load the CSV file from Notebook 1.


In [2]:
os.getenv("OPENAI_ORG_ID")

'org-U43gtMbXKtifsSUvWmXcbydP'

In [11]:
# Find the most recent CSV file in data/raw/
raw_data_dir = Path("../data/raw")
csv_files = list(raw_data_dir.glob("timeout_events_*.csv"))

if not csv_files:
    raise FileNotFoundError("No CSV files found in data/raw/. Please run Notebook 1 first.")

# Get the most recent file
latest_csv = sorted(csv_files)[-1]
print(f"Loading data from: {latest_csv}")

# Load data
df = pd.read_csv(latest_csv)

print(f"\n✅ Loaded {len(df)} events")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst event:")
df.head(1)


Loading data from: ../data/raw/timeout_events_20251107.csv

✅ Loaded 90 events

Columns: ['event_id', 'title', 'description', 'url', 'long_description', 'is_free']

First event:


,event_id,title,description,url,long_description,is_free
0,evt_001,The NY Comedy Festival,The New York Comedy Festival is where the best...,https://www.timeout.com/newyork/news/the-ny-co...,"The New York Comedy Festival(NYCF), the countr...",True


## 2. LLM-Based Feature Extraction

**✅✅✅ Simplified Approach:**
- Extract only `baby_friendly: bool` from event descriptions
- If True, implies stroller-accessible and suitable for infants/toddlers
- No mood tags or energy levels needed - semantic search handles this naturally!

**Why this works:**
- Baby-friendliness is objective and actionable
- Semantic embeddings capture nuanced vibes ("romantic", "relaxing", etc.) automatically
- Reduces API calls and processing time by 50%+


In [12]:
def extract_baby_friendly(title: str, description: str) -> dict:
    """
    Use GPT-4 to determine if an event is baby-friendly, indoor/outdoor, and create a clean summary.
    
    Args:
        title: Event title
        description: Event description
    
    Returns:
        dict: {"baby_friendly": bool, "indoor_or_outdoor": str, "clean_summary": str}
    """
    prompt = f"""Given this NYC event, provide three things:

1. Is it baby-friendly?
   - Suitable for infants and toddlers (ages 0-3)
   - Stroller-accessible
   - Family-oriented or welcoming to young children
   - Not explicitly adult-only (bars, nightclubs, R-rated content)

2. Is it indoor, outdoor, or both?
   - "indoor": Takes place entirely indoors (museums, theaters, indoor markets)
   - "outdoor": Takes place entirely outdoors (parks, outdoor festivals, street events)
   - "both": Has both indoor and outdoor components, or location is flexible

3. Create a clean, concise summary (MAX 5 sentences, ideally 2-3):
   - Remove all marketing language and promotional hype
   - Focus on WHAT the event is and what attendees will do/experience
   - Include key details: type of activity, notable features, what makes it unique
   - NO flowery language, NO calls to action, NO superlatives
   - Write in simple, factual sentences

Event Title: {title}
Event Description: {description}

Return ONLY a JSON object with this format:
{{
  "baby_friendly": true,
  "indoor_or_outdoor": "indoor",
  "clean_summary": "This is a brief 2-3 sentence summary of what the event actually is."
}}

Possible values for indoor_or_outdoor: "indoor", "outdoor", or "both"
No explanation needed, just the JSON."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Using mini for cost efficiency
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes event descriptions. Always respond with valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,  # Deterministic output
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        return {
            "baby_friendly": result.get("baby_friendly", False),
            "indoor_or_outdoor": result.get("indoor_or_outdoor", "indoor"),
            "clean_summary": result.get("clean_summary", f"{title}. {description[:200]}...")  # Fallback
        }
    
    except Exception as e:
        print(f"Error processing event: {e}")
        return {
            "baby_friendly": False,
            "indoor_or_outdoor": "indoor",
            "clean_summary": f"{title}. {description[:200]}..."
        }

print("✅ Function defined successfully!")

✅ Function defined successfully!


In [14]:
# Test the function on first event
test_event = df.iloc[0]
result = extract_baby_friendly(test_event['title'], test_event['description'])

print(f"Test event: {test_event['title']}")
print(f"Baby-friendly: {result['baby_friendly']}")
print(f"Indoor/Outdoor: {result['indoor_or_outdoor']}")
print(f"Clean description: {result['clean_summary']}")
print("\n✅ Test successful!")


Test event: The NY Comedy Festival
Baby-friendly: False
Indoor/Outdoor: indoor
Clean description: The New York Comedy Festival features over 200 comedians performing more than 100 shows across various venues in the five boroughs. Notable acts include comedy legends and a reunion of the cast from 'Strangers with Candy.' The festival runs from November 7 to November 16, 2025.

✅ Test successful!


In [17]:
# Parallelize event processing for faster extraction
# Using ThreadPoolExecutor for I/O-bound API calls
# Limit concurrent requests to avoid rate limits (8 concurrent is safe for OpenAI)

def process_event(row_tuple):
    """Wrapper function for parallel processing."""
    idx, row = row_tuple
    try:
        result = extract_baby_friendly(row['title'], row['long_description'])
        return idx, result
    except Exception as e:
        print(f"Error processing event {idx} ({row['title']}): {e}")
        return idx, {
            "baby_friendly": False,
            "indoor_or_outdoor": "indoor",
            "clean_summary": f"{row['title']}. {row['long_description'][:200]}..."
        }

# Prepare data for parallel processing
event_rows = [(idx, row) for idx, row in df.iterrows()]

# Process events in parallel with progress bar
print(f"Processing {len(event_rows)} events in parallel...")
print("Using 8 concurrent workers (adjust if you hit rate limits)\n")

baby_friendly_flags = [None] * len(df)
indoor_outdoor_flags = [None] * len(df)
clean_summaries = [None] * len(df)

# Use ThreadPoolExecutor for parallel API calls
with ThreadPoolExecutor(max_workers=8) as executor:
    # Submit all tasks
    future_to_idx = {executor.submit(process_event, row_tuple): row_tuple[0] 
                     for row_tuple in event_rows}
    
    # Process completed tasks with progress bar
    for future in tqdm(as_completed(future_to_idx), total=len(event_rows), desc="Processing events"):
        idx, result = future.result()
        baby_friendly_flags[idx] = result['baby_friendly']
        indoor_outdoor_flags[idx] = result['indoor_or_outdoor']
        clean_summaries[idx] = result['clean_summary']

# Add to dataframe
df['baby_friendly'] = baby_friendly_flags
df['indoor_or_outdoor'] = indoor_outdoor_flags
df['clean_summary'] = clean_summaries

print(f"\n✅ Processing complete!")
print(f"Baby-friendly events: {sum(baby_friendly_flags)} out of {len(df)} ({sum(baby_friendly_flags)/len(df)*100:.1f}%)")
print(f"\nIndoor/Outdoor breakdown:")
print(f"  Indoor: {indoor_outdoor_flags.count('indoor')}")
print(f"  Outdoor: {indoor_outdoor_flags.count('outdoor')}")
print(f"  Both: {indoor_outdoor_flags.count('both')}")

Processing 90 events in parallel...
Using 8 concurrent workers (adjust if you hit rate limits)



Processing events: 100%|██████████| 90/90 [00:31<00:00,  2.84it/s]


✅ Processing complete!
Baby-friendly events: 57 out of 90 (63.3%)

Indoor/Outdoor breakdown:
  Indoor: 64
  Outdoor: 13
  Both: 13


In [18]:
# Preview some baby-friendly events
baby_friendly_events = df[df['baby_friendly'] == True]
print(f"Sample baby-friendly events:")
baby_friendly_events[['title', 'baby_friendly']].head(10)


Sample baby-friendly events:


,title,baby_friendly
1,The Other Art Fair Brooklyn,True
2,Canstruction,True
3,Queer History Walking Tour,True
4,Cheese Week,True
6,FAD Market,True
8,"""CHROMA: Tales Between Hues"" immersive exhibit",True
9,"""Renoir’s Drawings"" at The Morgan",True
10,"""Rauschenberg’s New York: Pictures from the Re...",True
12,Grand Bazaar NYC,True
14,"""Monet and Venice"" at Brooklyn Museum",True


## 3. Create Embeddings

Generate semantic embeddings using OpenAI's `text-embedding-3-small` model.

**✅✅✅ Embedding Strategy:**
- Combine title + description for rich semantic representation
- Use OpenAI's latest embedding model (1536 dimensions)
- Event-level chunks: 1 event = 1 embedding


In [19]:
def create_embedding(text: str) -> list[float]:
    """
    Create embedding for text using OpenAI.
    
    Args:
        text: Text to embed
    
    Returns:
        list[float]: 1536-dimensional embedding vector
    """
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error creating embedding: {e}")
        return [0.0] * 1536  # Return zero vector on error

print("✅ Embedding function defined!")


✅ Embedding function defined!


In [20]:
# Create text for embedding (title + description)
df['embedding_text'] = df['title'] + " " + df['clean_summary']


# Generate embeddings for all events
print("Generating embeddings for all events...")
print("This will take ~1-2 minutes for 100 events.\n")

embeddings = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Creating embeddings"):
    embedding_text = f"{row['title']}. {row['clean_summary']}"
    print(embedding_text)
    embedding = create_embedding(row['embedding_text'])
    embeddings.append(embedding)
    
    # Rate limiting
    time.sleep(0.05)

# Convert to numpy array
embeddings_array = np.array(embeddings)

print(f"\n✅ Created {len(embeddings)} embeddings")
print(f"Embedding shape: {embeddings_array.shape}")


Generating embeddings for all events...
This will take ~1-2 minutes for 100 events.



Creating embeddings:   0%|          | 0/90 [00:00<?, ?it/s]

The NY Comedy Festival. The New York Comedy Festival features over 200 comedians performing across 100 shows at various iconic venues in NYC. The festival includes stand-up, improv, panels, and live podcast tapings. Notable events include a reunion of the cast from 'Strangers with Candy' and showcases for emerging talent.


Creating embeddings:   1%|          | 1/90 [00:00<00:55,  1.60it/s]

The Other Art Fair Brooklyn. The Other Art Fair Brooklyn features original artworks from over 130 independent and emerging artists. Attendees can meet artists and explore various media including painting, photography, and sculpture. The event includes interactive installations and immersive experiences.


...

The best fall activities in NYC to do with the arrival of Autumn. The event highlights various fall activities in NYC, including Halloween events and leaf-peeping opportunities. It focuses on experiences suitable for families and young children. Attendees can enjoy the seasonal atmosphere and participate in outdoor and indoor activities.


Creating embeddings:  99%|█████████▉| 89/90 [00:31<00:00,  3.03it/s]

The 50 best things to do in NYC for locals and tourists. This event highlights various activities and attractions in New York City suitable for both locals and tourists. It includes visits to art museums, Broadway shows, and dining experiences. The focus is on showcasing the best seasonal events and notable features of the city.


Creating embeddings: 100%|██████████| 90/90 [00:31<00:00,  2.85it/s]


✅ Created 90 embeddings
Embedding shape: (90, 1536)


## 4. Save Processed Data

Save the enriched data and embeddings to disk.


In [21]:
# Create processed directory if it doesn't exist
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# Save CSV (without embeddings - too large)
csv_path = processed_dir / "events_with_metadata.csv"
df_to_save = df.drop(columns=['embedding_text'])  # Remove temporary column
df_to_save.to_csv(csv_path, index=False)

# Save embeddings as numpy array
embeddings_path = processed_dir / "embeddings.npy"
np.save(embeddings_path, embeddings_array)

print(f"✅ Saved processed data to:")
print(f"  - CSV: {csv_path}")
print(f"  - Embeddings: {embeddings_path}")
print(f"\nFile sizes:")
print(f"  - CSV: {csv_path.stat().st_size / 1024:.1f} KB")
print(f"  - Embeddings: {embeddings_path.stat().st_size / 1024:.1f} KB")


✅ Saved processed data to:
  - CSV: ../data/processed/events_with_metadata.csv
  - Embeddings: ../data/processed/embeddings.npy

File sizes:
  - CSV: 309.3 KB
  - Embeddings: 1080.1 KB


## 5. Initialize Qdrant & Upload Events

Set up Qdrant vector database and upload all events with embeddings and metadata.

**✅✅✅ Vector Database Setup:**
- Using Qdrant local mode (no server needed)
- 1536-dimensional vectors (OpenAI embedding size)
- Cosine similarity for semantic search
- Metadata fields: event_id, title, description, url, baby_friendly


In [22]:
# Initialize Qdrant client (local mode)
qdrant_client = QdrantClient(path="../local_qdrant")

collection_name = "nyc_events"

# Delete collection if it exists (fresh start)
try:
    qdrant_client.delete_collection(collection_name)
    print(f"Deleted existing collection '{collection_name}'")
except:
    print(f"Collection '{collection_name}' doesn't exist yet")

# Create new collection
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=1536,  # OpenAI text-embedding-3-small dimension
        distance=Distance.COSINE
    )
)

print(f"\n✅ Created collection '{collection_name}'")


Deleted existing collection 'nyc_events'

✅ Created collection 'nyc_events'


In [23]:
processed_dir = Path("../data/processed")
embeddings_path = processed_dir / "embeddings.npy"

if embeddings_path.exists():
    print(f"✅ Loading embeddings from: {embeddings_path}")
    embeddings = np.load(embeddings_path)
    print(f"✅ Loaded {len(embeddings)} embeddings (shape: {embeddings.shape})")
    
    # Verify embeddings match dataframe length
    if len(embeddings) != len(df):
        raise ValueError(
            f"Embeddings count ({len(embeddings)}) doesn't match dataframe length ({len(df)}). "
            f"Please regenerate embeddings or load the correct dataframe."
        )
else:
    raise FileNotFoundError(
        f"Embeddings file not found at {embeddings_path}. "
        f"Please run the embedding generation cell first."
    )

✅ Loading embeddings from: ../data/processed/embeddings.npy
✅ Loaded 90 embeddings (shape: (90, 1536))


In [24]:
# Prepare points for upload
points = []

for idx, row in df.iterrows():
    point = PointStruct(
        id=idx,  # Use DataFrame index as ID
        vector=embeddings[idx],
        payload={
            "event_id": row['event_id'],
            "title": row['title'],
            "description": row['description'],
            "url": row['url'],
            "baby_friendly": bool(row['baby_friendly']),  # Ensure boolean type
            "is_free": bool(row['is_free']),  # Ensure boolean type
            "indoor_or_outdoor": str(row['indoor_or_outdoor'])  # "indoor", "outdoor", or "both"
        }
    )
    points.append(point)

# Upload to Qdrant
print(f"Uploading {len(points)} events to Qdrant...")
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"\n✅ Successfully uploaded {len(points)} events to Qdrant!")

# Verify collection info
collection_info = qdrant_client.get_collection(collection_name)
print(f"\nCollection info:")
print(f"  - Vectors count: {collection_info.vectors_count}")
print(f"  - Points count: {collection_info.points_count}")


Uploading 90 events to Qdrant...

✅ Successfully uploaded 90 events to Qdrant!

Collection info:
  - Vectors count: None
  - Points count: 90


## 6. Test Retrieval

Test semantic search and metadata filtering.


In [25]:
def search_events(query: str, top_k: int = 5, baby_friendly_only: bool = False, free_only: bool = False, indoor_or_outdoor: str = None):
    """
    Search for events using semantic similarity.
    
    Args:
        query: Search query
        top_k: Number of results to return
        baby_friendly_only: If True, only return baby-friendly events
        free_only: If True, only return free events
        indoor_or_outdoor: Filter by location type ("indoor", "outdoor", or "both")
    
    Returns:
        List of search results
    """
    # Create embedding for query
    query_embedding = create_embedding(query)
    
    # Prepare filter if needed
    search_filter = None
    if baby_friendly_only or free_only or indoor_or_outdoor:
        from qdrant_client.models import Filter, FieldCondition, MatchValue
        
        conditions = []
        if baby_friendly_only:
            conditions.append(
                FieldCondition(
                    key="baby_friendly",
                    match=MatchValue(value=True)
                )
            )
        if free_only:
            conditions.append(
                FieldCondition(
                    key="is_free",
                    match=MatchValue(value=True)
                )
            )
        if indoor_or_outdoor:
            conditions.append(
                FieldCondition(
                    key="indoor_or_outdoor",
                    match=MatchValue(value=indoor_or_outdoor)
                )
            )
        
        search_filter = Filter(must=conditions)
    
    # Search Qdrant
    results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        query_filter=search_filter,
        limit=top_k
    )
    
    return results

print("✅ Search function defined!")


✅ Search function defined!


In [26]:
# Test 1: General search
query1 = "fun outdoor activity for kids"
print(f"Query: '{query1}'\n")

results1 = search_events(query1, top_k=5)

for i, result in enumerate(results1, 1):
    print(f"{i}. {result.payload['title']}")
    print(f"   Score: {result.score:.3f}")
    print(f"   Baby-friendly: {result.payload['baby_friendly']}")
    print(f"   Description: {result.payload['description'][:100]}...\n")


Query: 'fun outdoor activity for kids'

1. Laser tag at mini-bowling at Area 53
   Score: 0.379
   Baby-friendly: True
   Description: Tucked away on Bridge Street in an old factory basement, this two-story playscape for kidsandadults ...

2. The best fall activities in NYC to do with the arrival of Autumn
   Score: 0.372
   Baby-friendly: True
   Description: The ultimate guide to fall in NYC, from leaf-peeping and apple picking to jack o' lantern festivals ...

3. “Gardens of Renewal” by Lily Kwong
   Score: 0.308
   Baby-friendly: True
   Description: When in need of a mental break, get yourself toMadison Square Parkto walk along a new meditative spi...

4. Wild Captives archery studio
   Score: 0.281
   Baby-friendly: False
   Description: Wild Captives, the nation’s first female- and LGBTQ-owned archery studio, is now open. It's a place ...

5. Torch & Crown's open-air beer garden
   Score: 0.273
   Baby-friendly: True
   Description: Torch & Crown Brewing Companyhas taken over pa

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_82944/2969955091.py:49: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [27]:
# Test 2: Baby-friendly filtered search
query2 = "art museum"
print(f"Query: '{query2}' (baby-friendly only)\n")

results2 = search_events(query2, top_k=5, baby_friendly_only=True)

for i, result in enumerate(results2, 1):
    print(f"{i}. {result.payload['title']}")
    print(f"   Score: {result.score:.3f}")
    print(f"   Baby-friendly: {result.payload['baby_friendly']}")
    print(f"   Description: {result.payload['description'][:100]}...\n")


Query: 'art museum' (baby-friendly only)

1. Arte Museum
   Score: 0.583
   Baby-friendly: True
   Description: Lose yourself in immersive digital art, evocative soundscapes and custom-crafted scents at the new A...

2. The newly renovated Rockefeller Wing at The Met
   Score: 0.440
   Baby-friendly: True
   Description: After a four-year renovation, The Metropolitan Museum of Art has reopened its galleries dedicated to...

3. MoMA Before Hours Tour
   Score: 0.435
   Baby-friendly: True
   Description: On a typical visit to theMuseum of Modern Art, crowds surround the most precious paintings, and it c...

4. The Other Art Fair Brooklyn
   Score: 0.418
   Baby-friendly: True
   Description: Connect with artists in-person and explore hundreds of original artworks across various media includ...

5. "Breaking the Mold: Brooklyn Museum at 200" exhibit
   Score: 0.410
   Baby-friendly: True
   Description: The Brooklyn Museum is celebrating a big birthday. As the museum turns 200, it’s mark

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_82944/2969955091.py:49: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [28]:
# Test 3: Semantic search (mood/vibe)
query3 = "romantic date night free"
print(f"Query: '{query3}'\n")
print("✅✅✅ Note: No explicit mood tags needed! Semantic search captures vibe naturally.\n")

results3 = search_events(query3, top_k=5)

for i, result in enumerate(results3, 1):
    print(f"{i}. {result.payload['title']}")
    print(f"   Score: {result.score:.3f}")
    print(f"   Baby-friendly: {result.payload['baby_friendly']}")
    print(f"   Description: {result.payload['description'][:100]}...\n")


Query: 'romantic date night free'

✅✅✅ Note: No explicit mood tags needed! Semantic search captures vibe naturally.

1. Friday Night Vibes at Time Out Market
   Score: 0.309
   Baby-friendly: False
   Description: Start your weekend off right at Time Out Market New York’s stunning rooftop! Friday Night Vibes gets...

2. Latin Mix Saturdays
   Score: 0.273
   Baby-friendly: False
   Description: Experience the ultimate Saturday night at Time Out Market’s Rooftop Latin Mix Party! Join resident D...

3. Rooftop Cinema Club
   Score: 0.272
   Baby-friendly: False
   Description: Rooftop Cinema Club takes movie-going to a whole new level—literally. This rooftop film series at a ...

4. LOOK Dine-in Cinemas
   Score: 0.265
   Baby-friendly: True
   Description: With a full restaurant, craft cocktails, comfy reclining seats and even more bells and whistles, thi...

5. Shake Rattle & Roll Dueling Pianos
   Score: 0.259
   Baby-friendly: False
   Description: Every Saturday night, two piano men

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_82944/2969955091.py:49: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [29]:
# Test 4: Free events only
query4 = "art museums and exhibitions"
print(f"Query: '{query4}' (free only)\n")
print("✅✅✅ Testing free_only filter to find budget-friendly events.\n")

results4 = search_events(query4, top_k=5, free_only=True)

for i, result in enumerate(results4, 1):
    print(f"{i}. {result.payload['title']}")
    print(f"   Score: {result.score:.3f}")
    print(f"   Free: {result.payload.get('is_free', 'N/A')}")
    print(f"   Baby-friendly: {result.payload['baby_friendly']}")
    print(f"   Description: {result.payload['description'][:100]}...\n")


Query: 'art museums and exhibitions' (free only)

✅✅✅ Testing free_only filter to find budget-friendly events.

1. "Renoir’s Drawings" at The Morgan
   Score: 0.472
   Free: True
   Baby-friendly: True
   Description: Renoir’s sketchbook is moving into the spotlight. TheMorgan Library & Museumis about to do something...

2. The newly renovated Rockefeller Wing at The Met
   Score: 0.463
   Free: True
   Baby-friendly: True
   Description: After a four-year renovation, The Metropolitan Museum of Art has reopened its galleries dedicated to...

3. "Man Ray: When Objects Dream" at the Met
   Score: 0.436
   Free: True
   Baby-friendly: True
   Description: On a cold winter day in 1921, artist Man Ray placed some of his glass equipment on top of an unexpos...

4. NYPL's free exhibit of rare artifacts
   Score: 0.416
   Free: True
   Baby-friendly: True
   Description: The New York Public Library dug through its expansive and centuries-spanning archive to stage an imp...

5. "Robert Rauschen

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_82944/2969955091.py:49: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [30]:
# Test 5: Combined filters - Baby-friendly AND Free
query5 = "family activities"
print(f"Query: '{query5}' (baby-friendly AND free)\n")
print("✅✅✅ Testing combined filters for budget-friendly family events.\n")

results5 = search_events(query5, top_k=5, baby_friendly_only=True, free_only=True)

if results5:
    for i, result in enumerate(results5, 1):
        print(f"{i}. {result.payload['title']}")
        print(f"   Score: {result.score:.3f}")
        print(f"   Free: {result.payload.get('is_free', 'N/A')}")
        print(f"   Baby-friendly: {result.payload['baby_friendly']}")
        print(f"   Description: {result.payload['description'][:100]}...\n")
else:
    print("No results found matching both criteria.")


Query: 'family activities' (baby-friendly AND free)

✅✅✅ Testing combined filters for budget-friendly family events.

1. The best fall activities in NYC to do with the arrival of Autumn
   Score: 0.338
   Free: True
   Baby-friendly: True
   Description: The ultimate guide to fall in NYC, from leaf-peeping and apple picking to jack o' lantern festivals ...

2. The 50 best things to do in NYC for locals and tourists
   Score: 0.273
   Free: True
   Baby-friendly: True
   Description: Every day, our staffers are eating, drinking, partying, gigging and generally appreciating their way...

3. FAD Market
   Score: 0.250
   Free: True
   Baby-friendly: True
   Description: Shop 'til you drop at FAD Market, a curated fashion, art and design pop-up marketplace, which is bac...

4. "Aunties" sculptures in Harlem
   Score: 0.246
   Free: True
   Baby-friendly: True
   Description: Three colorful figures are now brightening up the intersection of 124th Street and Lenox Avenue in H...

5. Reposo y

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_82944/2969955091.py:49: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [31]:
# Test 5: Combined filters - Baby-friendly AND Free
query6 = "scary"
print(f"Query: '{query6}' (baby-friendly AND free AND indoors)\n")
print("✅✅✅ Testing combined filters for budget-friendly family events.\n")

results6 = search_events(query6, top_k=5, baby_friendly_only=True, free_only=True, indoor_or_outdoor='indoor')

if results6:
    for i, result in enumerate(results6, 1):
        print(f"{i}. {result.payload['title']}")
        print(f"   Score: {result.score:.3f}")
        print(f"   Free: {result.payload.get('is_free', 'N/A')}")
        print(f"   Baby-friendly: {result.payload['baby_friendly']}")
        print(f"   Indoor or outdoor: {result.payload['indoor_or_outdoor']}")
        print(f"   Description: {result.payload['description'][:100]}...\n")
else:
    print("No results found matching both criteria.")


Query: 'scary' (baby-friendly AND free AND indoors)

✅✅✅ Testing combined filters for budget-friendly family events.

1. "CHROMA: Tales Between Hues" immersive exhibit
   Score: 0.147
   Free: True
   Baby-friendly: True
   Indoor or outdoor: indoor
   Description: Leave the gray of the city behind and step into a colorful world of Korean folktales at Genesis Hous...

2. "Dress, Dreams and Desire: Fashion and Psychoanalysis" at The Museum at FIT
   Score: 0.135
   Free: True
   Baby-friendly: True
   Indoor or outdoor: indoor
   Description: In the year 2025, how we dress is still the highest form of free self expression—and the role that g...

3. The Big Bubble Experiment at NYSCI
   Score: 0.133
   Free: True
   Baby-friendly: True
   Indoor or outdoor: indoor
   Description: Beautiful, buoyant, beguiling bubbles are back at theNew York Hall of Science (NYSCI)in Queens. The ...

4. Canstruction
   Score: 0.120
   Free: True
   Baby-friendly: True
   Indoor or outdoor: indoor
   Descr

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_82944/2969955091.py:49: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


## Summary & Next Steps

### ✅ Completed Tasks:

1. **Loaded raw data** from Notebook 1
2. **Extracted baby-friendly metadata** using GPT-4 Mini
   - Simple boolean flag: suitable for infants/toddlers
   - No mood or energy tags needed!
3. **Generated embeddings** using OpenAI text-embedding-3-small
   - 1536-dimensional vectors
   - Combined title + description for rich semantics
4. **Saved processed data**
   - CSV: `data/processed/events_with_metadata.csv`
   - Embeddings: `data/processed/embeddings.npy`
5. **Initialized Qdrant** vector database
   - Local mode (no server needed)
   - Cosine similarity search
6. **Uploaded all events** with metadata
7. **Tested retrieval** with multiple queries
   - General semantic search ✅
   - Baby-friendly filtering ✅
   - Mood/vibe queries ✅
8. **Created utility module** (`backend/vector_store.py`)

### 📊 Results:

- **Events processed:** Check cell output above
- **Baby-friendly events:** Check percentage above
- **Vector database:** Operational and ready
- **Search quality:** Semantic search captures nuance without explicit tags!

### 🎯 Next Steps:

**Move to Notebook 3:** Agentic RAG Pipeline
- Build 2-agent system (Retrieval + Response)
- Create FastAPI backend
- Implement LangSmith logging
- Test end-to-end queries

---

**✅✅✅ Key Insight:** By keeping metadata extraction simple (just baby-friendly), we saved significant time while still enabling powerful semantic search. The embeddings naturally capture mood, vibe, and context without explicit tagging!
